comparing all clustering, regressing methods: the forecast errors on raw data (withoug logarithm), it's remarkably 50% high... then we refine to find a narrow foracasting error


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection as b
from haversine import haversine
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans

In [ ]:
ds=pd.read_csv('../input/train.csv')[:10000]
test=pd.read_csv('../input/test.csv')[:10000]

In [ ]:
def dddraw(X_reduced,name):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    # To getter a better understanding of interaction of the dimensions
    # plot the first three PCA dimensions
    fig = plt.figure(1, figsize=(8, 6))
    ax = Axes3D(fig, elev=-150, azim=110)
    ax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=Y,cmap=plt.cm.Paired)
    titel="First three directions of "+name 
    ax.set_title(titel)
    ax.set_xlabel("1st eigenvector")
    ax.w_xaxis.set_ticklabels([])
    ax.set_ylabel("2nd eigenvector")
    ax.w_yaxis.set_ticklabels([])
    ax.set_zlabel("3rd eigenvector")
    ax.w_zaxis.set_ticklabels([])

    plt.show()

What do we see ?
---

* PCA shows 2x6 clusters grouped in two clusters, and forecast 7.9% error
* FastICa shows 2 clusters, didn't converge
* gauss is not so interesting 10% error
* KMeans has also that same 2x6 clusters, 8.9% error
* sparseRP separates in similar 2x6 clustersn, 7.9% error
* Birch has the same separation as Kmeans, 9.7% error
* NMF shows one big cloud, and evenhow gets 8.6%error



In [ ]:
from sklearn.decomposition import PCA, FastICA,SparsePCA,NMF, LatentDirichletAllocation,FactorAnalysis
from sklearn.random_projection import GaussianRandomProjection,SparseRandomProjection
from sklearn.cluster import KMeans,Birch
import statsmodels.formula.api as sm
from scipy import linalg
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler,PolynomialFeatures
def rmsle(y_predicted, y_real):
    return np.sqrt(np.mean(np.power(np.log1p(y_predicted)-np.log1p(y_real), 2)))
def procenterror(y_predicted, y_real):
     return ( np.mean(np.abs(y_predicted-y_real) )/ np.mean(y_real) *100 ).round()

n_col=28
X = ds.drop(['id','pickup_datetime','dropoff_datetime','store_and_fwd_flag','trip_duration'],axis=1) # we only take the first two features.
le = preprocessing.LabelEncoder()
def rmsle(y_predicted, y_real):
    return np.sqrt(np.mean(np.power(np.log1p(y_predicted)-np.log1p(y_real), 2)))
def procenterror(y_predicted, y_real):
     return np.round( np.mean(np.abs(y_predicted-y_real) )/ np.mean(y_real) *100 ,1)

    
#le.fit(np.round(ds['trip_duration']/30))
#print(list(le.classes_))
#Y=le.transform(np.round(ds['trip_duration']/30))
Y=np.round(np.log(ds['trip_duration'])*25)
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)
poly = PolynomialFeatures(2)
X=poly.fit_transform(X)


names = [
         'PCA',
         'FastICA',
         'Gauss',
         'KMeans',
         #'SparsePCA',
         'SparseRP',
         'Birch',
         'NMF',    
       #  'LatentDietrich',    
        ]

classifiers = [
    
    PCA(n_components=n_col),
    FastICA(n_components=n_col),
    GaussianRandomProjection(n_components=3),
    KMeans(n_clusters=10),
    #SparsePCA(n_components=n_col),
    SparseRandomProjection(n_components=n_col, dense_output=True),
    Birch(branching_factor=10, n_clusters=3, threshold=0.5),
    NMF(n_components=n_col),    
  #  LatentDirichletAllocation(n_topics=n_col),
    
]
correction= [1,1,0,0,0,0,0,0,0]

temp=zip(names,classifiers,correction)
print(temp)

for name, clf,correct in temp:
    Xr=clf.fit_transform(X,Y)
    dddraw(Xr,name)
    res = sm.OLS(Y,Xr).fit()
    #print(res.summary())  # show OLS regression
    #print(res.predict(Xr).round()+correct)  #show OLS prediction
    #print('Ypredict',res.predict(Xr).round()+correct)  #show OLS prediction
    
    print('Ypredict',res.predict(Xr).round()+correct*Y.mean())  #show OLS prediction
    print(name,'%error',procenterror(res.predict(Xr)+correct*Y.mean(),Y),'rmsle',rmsle(res.predict(Xr)+correct*Y.mean(),Y)) #
    
    
    
    

In [ ]:
pl=np.array(ds['pickup_latitude'])
plo=np.array(ds['pickup_longitude'])
dl=np.array(ds['dropoff_latitude'])
dlo=np.array(ds['dropoff_longitude'])
pl=pl.reshape(-1,1)
plo=plo.reshape(-1,1)
dl=dl.reshape(-1,1)
dlo=dlo.reshape(-1,1)
m = pl.size
n = 4
xk = np.empty([m,n], float)
xk[:,0]=pl[:,0]
xk[:,1]=plo[:,0]
xk[:,2]=dl[:,0]
xk[:,3]=dlo[:,0]

Lets cluster in 24 groups 
---
even though there are 12 groups visible in the clusters it gives some redundancy. And we try to find a statistical tripduration/standardevation, and efficiency clustering
Its simply a grouping of start-stop places and helps to forecast the typical speed for such a distance



In [ ]:
from geopy.distance import vincenty
ds['geo_distance']=ds.apply(lambda x: vincenty((x['pickup_latitude'] ,x['pickup_longitude']),(x['dropoff_latitude'], x['dropoff_longitude'])).miles,axis=1)
test['geo_distance']=test.apply(lambda x: vincenty((x['pickup_latitude'] ,x['pickup_longitude']),(x['dropoff_latitude'], x['dropoff_longitude'])).miles,axis=1)
ds['time_per_km']=np.log(ds['trip_duration']/(ds['geo_distance']/60+0.0001)+1)

In [ ]:
kmeans = KMeans(n_clusters=48, random_state=0).fit(xk)
ds['pickup_id']=kmeans.labels_

In [ ]:
def clust(x):
    kl=0
    if x<0.3:
        kl=1
    if x>0.29 and x<0.6:
        kl=2
    if x>0.59:
        kl=4
    return kl
ds['log_trip_duration']=np.log(ds['trip_duration'])
new_col= ds[['time_per_km','log_trip_duration','pickup_id']].groupby('pickup_id').describe().fillna(method='bfill')
new_col.columns=['countt','meant','stdt','mint','p25t','p50t','p75t','maxt','countgt','meangt','stdgt','mingt','p25gt','p50gt','p75gt','maxgt']
new_col['efft']=new_col['stdt']/new_col['meant']
new_col['eff2t']=new_col['efft']*new_col['stdt']
new_col['clustt']=new_col['eff2t'].map(clust)
print(new_col.head())
ds=pd.merge(ds,new_col, how='outer', left_on='pickup_id',suffixes=('', '_c'), right_index=True)

HourofWeek
----
Another grouping statistics. based upon the dayofweek and hour of day combination. There are 168hours per week. And each hour has his average speed and efficiency properties


In [ ]:
ds['pickup_datetime']=pd.to_datetime(ds['pickup_datetime'])
test['pickup_datetime']=pd.to_datetime(test['pickup_datetime'])

ds['pickup_weekday']=ds['pickup_datetime'].dt.weekday
ds['pickup_hour']=ds['pickup_datetime'].dt.hour
ds['pickup_month']=ds['pickup_datetime'].dt.month
ds['pickup_weekhour']=ds['pickup_weekday']*24+ds['pickup_hour']

new_col= ds[['log_trip_duration','time_per_km','pickup_weekhour']].groupby('pickup_weekhour').describe().fillna(method='bfill')
new_col.columns=['countt','meant','stdt','mint','p25t','p50t','p75t','maxt','countgt','meangt','stdgt','mingt','p25gt','p50gt','p75gt','maxgt']
new_col['efft']=new_col['stdt']/new_col['meant']
new_col['eff2t']=new_col['efft']*new_col['stdt']
new_col['clustt']=new_col['eff2t'].map(clust)
print(new_col.head())
ds=pd.merge(ds,new_col, how='outer', left_on='pickup_weekhour',suffixes=('', '_wh'), right_index=True)

In [ ]:
#ds['distance']=ds.apply(lambda x: haversine((x['pickup_latitude'] ,x['pickup_longitude']),(x['dropoff_latitude'], x['dropoff_longitude']),miles=True),axis=1)
#test['distance']=test.apply(lambda x: haversine((x['pickup_latitude'] ,x['pickup_longitude']),(x['dropoff_latitude'], x['dropoff_longitude']),miles=True),axis=1)

The distance and the average time per distance
---

In [ ]:
loc_df = pd.DataFrame()
loc_df['longitude'] = list(ds.pickup_longitude) + list(ds.dropoff_longitude)
loc_df['latitude'] = list(ds.pickup_latitude) + list(ds.dropoff_latitude)
kmeans = KMeans(n_clusters=24, random_state=2, n_init = 10).fit(loc_df)
loc_df['label'] = kmeans.labels_
ds['pickup_cluster'] = kmeans.predict(ds[['pickup_longitude','pickup_latitude']])
ds['dropoff_cluster'] = kmeans.predict(ds[['dropoff_longitude','dropoff_latitude']])
test['pickup_cluster'] = kmeans.predict(test[['pickup_longitude','pickup_latitude']])
test['dropoff_cluster'] = kmeans.predict(test[['dropoff_longitude','dropoff_latitude']])

In [ ]:
test['pickup_weekday']=test['pickup_datetime'].dt.weekday
test['pickup_hour']=test['pickup_datetime'].dt.hour
test['pickup_month']=test['pickup_datetime'].dt.month
test['pickup_weekhour']=test['pickup_weekday']*24+test['pickup_hour']

In [ ]:
test['isweekend']= test.apply(lambda x : (x['pickup_weekday']==6 | x['pickup_weekday']==5),axis=1)
test['isweekend']=test['isweekend'].map({True: 1, False:0})
test['store_and_fwd_flag']=test['store_and_fwd_flag'].map({'N': 1, 'Y':0})

In [ ]:
Ypl = np.round(np.log(ds['trip_duration']))
plt.figure()
Ypl.plot.hist(alpha=0.5)


plt.show()

Clustering again shows smaller error
----
* PCA and FastIca improves to 6.6% error
* Gauss is worse
* Kmeans, improves to 7%, SparseRP improves minimally



In [ ]:
from sklearn.decomposition import PCA, FastICA,SparsePCA,NMF, LatentDirichletAllocation,FactorAnalysis
from sklearn.random_projection import GaussianRandomProjection,SparseRandomProjection
from sklearn.cluster import KMeans,Birch
import statsmodels.formula.api as sm
from scipy import linalg
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler,PolynomialFeatures
import matplotlib.pyplot as plt

def rmsle(y_predicted, y_real):
    return np.sqrt(np.mean(np.power(np.log1p(y_predicted)-np.log1p(y_real), 2)))
def procenterror(y_predicted, y_real):
     return ( np.mean(np.abs(y_predicted-y_real) )/ np.mean(y_real) *100 ).round()

n_col=28
X = ds.drop(['id','pickup_datetime','dropoff_datetime','store_and_fwd_flag','trip_duration','log_trip_duration','time_per_km'],axis=1) # we only take the first two features.
le = preprocessing.LabelEncoder()
def rmsle(y_predicted, y_real):
    return np.sqrt(np.mean(np.power(np.log1p(y_predicted)-np.log1p(y_real), 2)))
def procenterror(y_predicted, y_real):
     return np.round( np.mean(np.abs(y_predicted-y_real) )/ np.mean(y_real) *100 ,1)

    
le.fit(np.round(np.log(ds['trip_duration'])*10))

Y=le.transform(np.round(np.log(ds['trip_duration'])*10))

Y=np.round(np.log(ds['trip_duration'])*10)
               
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)
poly = PolynomialFeatures(2)
X=poly.fit_transform(X)


names = [
         'PCA',
         #'FastICA',
         'Gauss',
         'KMeans',
          #'SparsePCA',
         'SparseRP',
         #'Birch',
      #   'NMF',    
      #   'LatentDietrich',    
        ]

classifiers = [
    
    PCA(n_components=n_col),
   # FastICA(n_components=n_col),
    GaussianRandomProjection(n_components=3),
    KMeans(n_clusters=24),
 #   SparsePCA(n_components=n_col),
    SparseRandomProjection(n_components=n_col, dense_output=True),
  #  Birch(branching_factor=10, n_clusters=12, threshold=0.5),
  #  NMF(n_components=n_col),    
 #   LatentDirichletAllocation(n_topics=n_col),
    
]
correction= [1,1,0,0,0,0,0,0,0]

temp=zip(names,classifiers,correction)
print(temp)

for name, clf,correct in temp:
    Xr=clf.fit_transform(X,Y)
    dddraw(Xr,name)
    res = sm.OLS(Y,Xr).fit()
    #print(res.summary())  # show OLS regression
    #print(res.predict(Xr).round()+correct)  #show OLS prediction
    print('Ypredict',res.predict(Xr).round()+correct)  #show OLS prediction

    print('Ypredict *log_sec',res.predict(Xr).round()+correct*Y.mean())  #show OLS prediction
    print(name,'%error',procenterror(res.predict(Xr)+correct*Y.mean(),Y),'rmsle',rmsle(res.predict(Xr)+correct*Y.mean(),Y))

linear models forecast with lowest error for time being
----

* ElasticNet %error 6.24 rmsle 0.104493207131
* HuberRegressor %error 6.06 rmsle 0.103178076065
* Ridge %error 5.91 rmsle 0.0995130160778
* Lasso %error 6.43 rmsle 0.107492482631
* LassoCV %error 6.01 rmsle 0.100907758631
* Lars %error 6.45 rmsle 0.107083059727
* BayesianRidge %error 5.92 rmsle 0.0996487967579
* SGDClassifier %error 10.1 rmsle 0.174949522616
* RidgeClassifier %error 7.13 rmsle 0.123412139794
* LogisticRegression %error 7.03 rmsle 0.116931520745
* OrthogonalMatchingPursuit %error 6.31 rmsle 0.104854612413

In [ ]:
from sklearn.linear_model import OrthogonalMatchingPursuit,RANSACRegressor,LogisticRegression,ElasticNetCV,HuberRegressor, Ridge, Lasso,LassoCV,Lars,BayesianRidge,SGDClassifier,LogisticRegressionCV,RidgeClassifier
from sklearn.preprocessing import MinMaxScaler

# import some data to play with
n_col=50
X = ds.drop(['id','pickup_datetime','dropoff_datetime','store_and_fwd_flag','time_per_km','log_trip_duration','trip_duration'],axis=1) # we only take the first two features.
Y=np.round(ds['trip_duration']/30)
Y=np.round(np.log(ds['trip_duration'])*10)

scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)


def rmsle(y_predicted, y_real):
    return np.sqrt(np.mean(np.power(np.log1p(y_predicted)-np.log1p(y_real), 2)))
def procenterror(y_predicted, y_real):
     return np.round( np.mean(np.abs(y_predicted-y_real) )/ np.mean(y_real) *100 ,2)

names = [
         'ElasticNet',
         'HuberRegressor',
         'Ridge',
         'Lasso',
         'LassoCV',
         'Lars',
         'BayesianRidge',
         'SGDClassifier',
         'RidgeClassifier',
         'LogisticRegression',
         'OrthogonalMatchingPursuit',
         #'RANSACRegressor',
         ]

classifiers = [
    ElasticNetCV(cv=10, random_state=0),
    HuberRegressor(fit_intercept=True, alpha=0.0, max_iter=100,epsilon=2.95),
    Ridge(fit_intercept=True, alpha=0.0, random_state=0, normalize=True),
    Lasso(alpha=0.05),
    LassoCV(),
    Lars(n_nonzero_coefs=10),
    BayesianRidge(),
    SGDClassifier(),
    RidgeClassifier(),
    LogisticRegression(),
    OrthogonalMatchingPursuit(),
    #RANSACRegressor(),
]
correction= [0,0,0,0,0,0,0,0,0,0,0,0]

temp=zip(names,classifiers,correction)
print(temp)

for name, clf,correct in temp:
    regr=clf.fit(X,Y)
    #print( name,'% errors', abs(regr.predict(X)+correct-Y).sum()/(Y.sum())*100)
    print(name,'%error',procenterror(regr.predict(X),Y),'rmsle',rmsle(regr.predict(X),Y))


XGB superiority 
----
error 3.6%

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans

#train1=total[total['split']==0]
#test1=total[total['split']==1]
X = ds.drop(['id','pickup_datetime','dropoff_datetime','store_and_fwd_flag','time_per_km','log_trip_duration','trip_duration'],axis=1) # we only take the first two features.
Y=np.round(np.log(ds['trip_duration'])*10)
poly = PolynomialFeatures(2)
X=poly.fit_transform(X)

Xtr, Xv, ytr, yv = train_test_split(X, Y, test_size=0.2, random_state=2017)
dtrain = xgb.DMatrix(Xtr, label=ytr)
dvalid = xgb.DMatrix(Xv, label=yv)
#dtest = xgb.DMatrix(test1[feature_names].values)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

# Try different parameters! My favorite is random search :)
xgb_pars = {'min_child_weight': 100, 'eta': 0.03, 'colsample_bytree': 0.3, 'max_depth': 10,
            'subsample': 0.8, 'lambda': 1., 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:linear'}

# You could try to train with more epoch
model = xgb.train(xgb_pars, dtrain, 2000, watchlist, early_stopping_rounds=25,
                  maximize=False, verbose_eval=50)

In [ ]:
print('XGB %error',procenterror(model.predict(dtrain),ytr),'rmsle',rmsle(model.predict(dtrain),ytr))
